In [1]:
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Dropout, Reshape, Permute, Activation, BatchNormalization, \
    Input
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, Callback
import numpy as np
from keras.wrappers.scikit_learn import KerasRegressor
from keras.applications.resnet50 import ResNet50
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from image_util import crop_center
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [2]:
# validation keras fix 
# ref: http://stackoverflow.com/questions/41796618/python-keras-cross-val-score-error/41841066#41841066
from keras.wrappers.scikit_learn import BaseWrapper
import copy

def custom_get_params(self, **params):
    res = copy.deepcopy(self.sk_params)
    res.update({'build_fn': self.build_fn})
    return res

BaseWrapper.get_params = custom_get_params


In [ ]:
# load data
Xtrain = np.load('./data/dataset256/Xtrain.npy')[:, 16:240, 16:240, :]
Xtest = np.load('./data/dataset256/Xtest.npy')[:, 16:240, 16:240, :]
Ytrain = np.load('./data/dataset256/Ytrain.npy')
Ytest = np.load('./data/dataset256/Ytest.npy')

In [ ]:
Xtrain.shape

In [3]:
# netowrk settings
batch_size = 128
nb_epoch = 2

In [4]:
def ResNetTransfer_first_attempt():
    inp = Input(shape=(224,224,3))
    resnet_model = ResNet50(include_top=False, weights='imagenet', input_tensor=inp)
    
    out_model = Sequential()
    out_model.add(resnet_model)
    out_model.add(Flatten())
    out_model.add(Dense(256, activation='relu', name='fcc'))
    out_model.add(BatchNormalization())
    out_model.add(Dense(1, name='out'))

    out_model.compile(loss='mean_squared_error',
                  optimizer='adam')
    return out_model


In [5]:
def ResNetTransfer():
    inp = Input(shape=(224,224,3))
    layer = ResNet50(include_top=False, weights='imagenet', input_tensor=inp).outputs
    layer = Flatten()(layer)
    layer = Dense(256, activation='relu', name='fcc')(layer)
    layer = BatchNormalization()(layer)
    out = Dense(1, name='out')(layer)
    
    model = Model(input=inp, output=out)
    model.compile(loss='mean_squared_error',
                  optimizer='adam')
    return model


In [6]:
model = ResNetTransfer()
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 230, 230, 3)   0           input_1[0][0]                    
____________________________________________________________________________________________________
conv1 (Convolution2D)            (None, 112, 112, 64)  9472        zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
bn_conv1 (BatchNormalization)    (None, 112, 112, 64)  256         conv1[0][0]                      
___________________________________________________________________________________________

In [ ]:
# batch logger 
# ref: https://github.com/fchollet/keras/issues/2850#issuecomment-222542429
class NBatchLogger(Callback):
    def __init__(self,display=100):
        '''
        display: Number of batches to wait before outputting loss
        '''
        self.seen = 0
        self.display = display

    def on_batch_end(self,batch,logs={}):
        self.seen += logs.get('size', 0)
        if self.seen % self.display == 0:
            print('\n{0}/{1} - Batch Loss: {2}'.format(self.seen,self.params['nb_sample'],
                                                self.params['metrics'][0]))

In [ ]:
#train just 1000 samples, 3 epochs
# if False:
checkpoint = ModelCheckpoint(filepath='./training-checkpoint/checkpoint-{epoch:02d}-{val_loss:.2f}.hdf5')
out_batch = NBatchLogger(display=1)
history = model.fit(x=Xtrain[:256], y=Ytrain[:256], 
          nb_epoch=nb_epoch, batch_size=batch_size,validation_split=0.1, verbose=1, callbacks=[checkpoint, out_batch]) 

In [ ]:
plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

In [ ]:
model.evaluate(x=Xtest, y=Ytest,batch_size=batch_size)

In [ ]:
# regression 
# rev: http://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/

In [ ]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)
# evaluate model with standardized dataset
estimator = KerasRegressor(build_fn=MyNet, nb_epoch=nb_epoch, batch_size=batch_size, verbose=1)

In [ ]:
kfold = KFold(n_splits=10, random_state=seed)
results = cross_val_score(estimator, Xtrain[:1000], Ytrain[:1000], cv=kfold)
print("Results: %.2f (%.2f) MSE" % (results.mean(), results.std()))

In [ ]:
model.